In [4]:
import torch
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
model_path= 'models/pytorch/Pytorch_trial_weights.pt'

In [16]:
model = torch.load(model_path,map_location='cuda')
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.BatchNorm1d(512),  # Batch normalization layer
    nn.ReLU(inplace=True),  # activation function
    nn.Linear(512, 8),
    nn.Softmax(dim=1)
).to(device)

model.eval()

# Sample input (adjust according to your model's input shape)
dummy_input = torch.randn(1, 3, 224, 224).cuda()

torch.onnx.export(model, dummy_input, "resnet_gpu.onnx", opset_version=11)


In [12]:
pip install onnxruntime

   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.6 MB 660.6 kB/s eta 0:00:09
    --------------------------------------- 0.1/5.6 MB 1.3 MB/s eta 0:00:05
   - -------------------------------------- 0.3/5.6 MB 2.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.5/5.6 MB 3.3 MB/s eta 0:00:02
   --- ----------

In [17]:
import onnx
from onnxruntime import InferenceSession

# Load the ONNX model
model = onnx.load("resnet_cpu.onnx")

# Print input details
print("Input Name:", model.graph.input[0].name)
print("Input Shape:", model.graph.input[0].type.tensor_type.shape)

# Create an InferenceSession (optional)
session = InferenceSession(model.SerializeToString())
input_names = [input.name for input in session.get_inputs()]
input_shapes = [session.get_inputs()[i].shape for i in range(len(input_names))]
print("Input Names (from InferenceSession):", input_names)
print("Input Shapes (from InferenceSession):", input_shapes)

# Print layer details (optional)
for node in model.graph.node:
    print("Layer Name:", node.name)
    print("Layer Operator:", node.op_type)
    print("Layer Inputs:", node.input)
    print("Layer Outputs:", node.output)
    print("----------")


Input Name: input.1
Input Shape: dim {
  dim_value: 1
}
dim {
  dim_value: 3
}
dim {
  dim_value: 224
}
dim {
  dim_value: 224
}

Input Names (from InferenceSession): ['input.1']
Input Shapes (from InferenceSession): [[1, 3, 224, 224]]
Layer Name: Identity_0
Layer Operator: Identity
Layer Inputs: ['fc.1.weight']
Layer Outputs: ['fc.1.running_var']
----------
Layer Name: Identity_1
Layer Operator: Identity
Layer Inputs: ['fc.1.bias']
Layer Outputs: ['fc.1.running_mean']
----------
Layer Name: /conv1/Conv
Layer Operator: Conv
Layer Inputs: ['input.1', 'onnx::Conv_508', 'onnx::Conv_509']
Layer Outputs: ['/conv1/Conv_output_0']
----------
Layer Name: /relu/Relu
Layer Operator: Relu
Layer Inputs: ['/conv1/Conv_output_0']
Layer Outputs: ['/relu/Relu_output_0']
----------
Layer Name: /maxpool/MaxPool
Layer Operator: MaxPool
Layer Inputs: ['/relu/Relu_output_0']
Layer Outputs: ['/maxpool/MaxPool_output_0']
----------
Layer Name: /layer1/layer1.0/conv1/Conv
Layer Operator: Conv
Layer Inputs: ['

In [6]:
import onnx



In [11]:
import tensorrt as trt
import ctypes

# Load the serialized TensorRT model (.trt file)
with open("resnetGPU_engine.trt", "rb") as f:
    trt_engine_data = f.read()

# Load TensorRT runtime
trt_logger = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(trt_logger)

# Deserialize the TensorRT engine
trt_engine = trt_runtime.deserialize_cuda_engine(trt_engine_data)
print(trt_engine)

# Save the engine to file (assuming you want to save a copy)
try:
    with open("resnet_engine.engine", "wb") as f:
        f.write(trt_engine.serialize())
except Exception as e:
    print(f"Error saving engine: {e}")
    # Handle the error gracefully (e.g., exit program, log error)


None
Error saving engine: 'NoneType' object has no attribute 'serialize'


In [ ]:
import onnx
# Load the ONNX model

onnx_model_path = "resnet50_model.onnx"
onnx_model = onnx.load(onnx_model_path)

# Create a TensorRT builder and network definition
builder = trt.Builder(trt.Logger(trt.Logger.WARNING))
network = builder.create_network()

# Parse the ONNX model into the TensorRT network
parser = trt.OnnxParser(network, trt.Logger(trt.Logger.WARNING))
parser.parse(onnx_model)

# Set the maximum workspace size
workspace_size = 1 << 30  # 1GB
builder.set_workspace(workspace_size)

# Build the TensorRT engine
builder.max_batch_size = 1
engine = builder.build_cuda_engine(network)

# Save the TensorRT engine
trt_engine_path = "C:/Users/AS-GP/Desktop/Resnet50"
with open(trt_engine_path, "wb") as f:
    f.write(engine.serialize())


In [ ]:
from torch.utils.data import random_split

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])
}
full_dataset = datasets.ImageFolder(input_path + 
                                    '/train', 
                                    data_transforms['train'])

# Calculate the sizes of the training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders for training and validation sets
dataloaders = {
    'train': torch.utils.data.DataLoader(train_dataset,
                                         batch_size=32,
                                         shuffle=True,
                                         num_workers=2),
    'validation': torch.utils.data.DataLoader(val_dataset,
                                              batch_size=32,
                                              shuffle=True,
                                              num_workers=2)
}
print(len(train_dataset))
print(len(val_dataset))